# Tools

In [1]:
import os, numpy as np, glob
from matplotlib import pyplot as plt
%matplotlib notebook

In [2]:
import mcvine
from mccomponents.sample import matter
from sampleassembly.visitors.CrossSectionCalculator import CrossSectionCalculator
from mcvine.workflow.sampleassembly import scaffolding
from mcvine.workflow.sample import loadSampleYml

In [3]:
from mcvine.workflow.sampleassembly.scaffolding import sample, phonon_powder_incoherent, _kernel_helpers
import imp
imp.reload(sample)
imp.reload(phonon_powder_incoherent)
imp.reload(_kernel_helpers)

<module 'mcvine.workflow.sampleassembly.scaffolding._kernel_helpers' from '/home/97n/dv/mcvine/export-3.7/lib/python3.7/site-packages/mcvine/workflow/sampleassembly/scaffolding/_kernel_helpers.py'>

# Al crystal

In [4]:
ls Al*.cif

Al_mp-134_symmetrized.cif


In [5]:
!head -n 10 Al_mp-134_symmetrized.cif

# generated using pymatgen
data_Al
_symmetry_space_group_name_H-M   Fm-3m
_cell_length_a   4.03893000
_cell_length_b   4.03893000
_cell_length_c   4.03893000
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   225


## Common information that might be useful

In [6]:
structure = matter.loadCif('./Al_mp-134_symmetrized.cif')

In [7]:
xscalc = CrossSectionCalculator()
xscalc.include_density = False
print("abs (2200m/s), inc, coh cross sections = ", xscalc.onUnitCell(structure))
xscalc.include_density = True
print("abs (2200m/s), inc, coh inverse length =", xscalc.onUnitCell(structure))

abs (2200m/s), inc, coh cross sections =  [9.24e-29*m**2 3.28e-30*m**2 5.98e-28*m**2]
abs (2200m/s), inc, coh inverse length = [1.4024*m**-1 0.0497823*m**-1 9.07616*m**-1]


# Sample with Incoherent Elastic Kernel

In [8]:
!rm -rf Al-inc-el/
!mkdir -p Al-inc-el

## Yaml spec

In [9]:
%%file Al-inc-el.yaml
name: Al
structure_file: Al_mp-134_symmetrized.cif
excitation:
 type: powder_elastic_incoherent
 dw_core: 0.0*angstrom**2
 scattering_xs: 0.033*barn
 absorption_xs: 0.92*barn
shape: cylinder radius="1*mm" height="3*mm"
temperature: 300*K

Overwriting Al-inc-el.yaml


## Convert to xml

In [10]:
spec = loadSampleYml('Al-inc-el.yaml')
scaffolding.createSampleAssembly('Al-inc-el', spec, add_elastic_line=False)

/home/97n/dv/mcvine/export-3.7/lib/python3.7/site-packages/mcvine/workflow/sampleassembly/scaffolding/sampleassembly_xml.py:22: UserWarning: assume average temperature
  warnings.warn("assume average temperature")


In [11]:
ls Al-inc-el

Al_mp-134_symmetrized.cif  Al-scatterer.xml  sampleassembly.xml


In [12]:
!mcvine sampleassembly check Al-inc-el/sampleassembly.xml

cartesian coordinates of atom: [0. 0. 0.]
cartesian coordinates of atom: [2.019465 2.019465 0.      ]
cartesian coordinates of atom: [2.019465 0.       2.019465]
cartesian coordinates of atom: [0.       2.019465 2.019465]
weight:  1.0


# Sample with Incoherent Inelastic Phonon Kernel

In [13]:
!rm -rf Al-inc-inel/
!mkdir -p Al-inc-inel

## Yaml spec

In [14]:
%%file Al-inc-inel.yaml
name: Al
structure_file: Al_mp-134_symmetrized.cif
excitation:
 type: phonon_powder_incoherent
 DOS: Al-dos.txt
shape: cylinder radius="1*mm" height="3*mm"
temperature: 300*K

Overwriting Al-inc-inel.yaml


## Convert to xml

In [15]:
spec = loadSampleYml('Al-inc-inel.yaml')
scaffolding.createSampleAssembly('Al-inc-inel', spec, add_elastic_line=False)

Copying Al-dos.txt to Al-inc-inel/


In [16]:
ls Al-inc-inel

Al-dos.txt  Al_mp-134_symmetrized.cif  Al-scatterer.xml  sampleassembly.xml


In [17]:
!mcvine sampleassembly check Al-inc-inel/sampleassembly.xml

DOS: fit first 100 points to parbolic
cartesian coordinates of atom: [0. 0. 0.]
cartesian coordinates of atom: [2.019465 2.019465 0.      ]
cartesian coordinates of atom: [2.019465 0.       2.019465]
cartesian coordinates of atom: [0.       2.019465 2.019465]
weight:  1.0


# Sample with Powder Diffraction Kernel

In [18]:
!rm -rf Al-powder-diffraction
!mkdir -p Al-powder-diffraction

## Peaks python file

* If you have a laz file, you may want to use that.
* Shown here is a quick way to create diffraction peaks from crystal structure

In [19]:
from mccomponents.sample import diffraction

In [20]:
diffraction.create_peaks_py('Al-diffr-peaks.py', structure, 300., max_index=20, min_dspacing=.5)

/home/97n/dv/mcvine/export-3.7/lib/python3.7/site-packages/mccomponents/sample/diffraction/calcpeaks.py:85: RuntimeWarning: divide by zero encountered in double_scalars
  return 1./np.linalg.norm(recvec)


In [21]:
!head -n 10 Al-diffr-peaks.py

from mccomponents.sample.diffraction.SimplePowderDiffractionKernel import Peak

peaks = [
    Peak(hkl=(1, 1, 1), d=2.331877322738055, q=2.6944750677544076, F=(13.336929215115559-2.4499541510619136e-15j), F_squared=1.778736808890029, multiplicity=8, intrinsic_line_width=0, DebyeWaller_factor=0),
    Peak(hkl=(0, 0, 2), d=2.019465, q=3.1113118113854843, F=(13.187325715565134-1.614981622688042e-15j), F_squared=1.7390555952840545, multiplicity=6, intrinsic_line_width=0, DebyeWaller_factor=0),
    Peak(hkl=(0, 2, 2), d=1.427977395868891, q=4.400059360432953, F=(12.605505909568386-3.0874584927571937e-15j), F_squared=1.5889877923616351, multiplicity=12, intrinsic_line_width=0, DebyeWaller_factor=0),
    Peak(hkl=(1, 1, 3), d=1.2177832149554666, q=5.15952694208333, F=(12.186049582243726-3.73090165378643e-15j), F_squared=1.4849980442090247, multiplicity=24, intrinsic_line_width=0, DebyeWaller_factor=0),
    Peak(hkl=(2, 2, 2), d=1.1659386613690275, q=5.388950135508815, F=(12.04935577261231-4.4

## Yaml spec

In [22]:
%%file Al-powder-diffraction.yaml
name: Al
structure_file: Al_mp-134_symmetrized.cif
excitation:
 type: powder_diffraction
 Dd_over_d: 1e-5
 peaks_py_path: Al-diffr-peaks.py
shape: cylinder radius="1*mm" height="3*mm"
temperature: 300*K

Overwriting Al-powder-diffraction.yaml


## Convert to xml

In [23]:
spec = loadSampleYml('Al-powder-diffraction.yaml')
scaffolding.createSampleAssembly('Al-powder-diffraction', spec, add_elastic_line=False)

Copying Al-diffr-peaks.py to Al-powder-diffraction/


In [24]:
ls Al-powder-diffraction/

Al-diffr-peaks.py          Al-scatterer.xml
Al_mp-134_symmetrized.cif  sampleassembly.xml


In [25]:
!mcvine sampleassembly check Al-powder-diffraction//sampleassembly.xml

weight:  1.0
